# Питон и машинное обучение

# Модуль 9a. Метод поиска ближайших соседей



In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

matplotlib.rcParams['figure.figsize'] = (14,9)

%matplotlib inline

## Классификация объектов по бесконечному количеству классов методом поиска ближайших соседей

Задача классификации по бесконечному количеству классов возникает в следующих случаях:
- подбор похожих объектов по образцу:
    - рекомендательные системы
    - поиск фото по текстовому описанию и по фото
    - поиск и идентификация аномалий на рентгеновских снимках/МРТ/КТ
- идентификация объектов по различным паттернам:
    - поиск людей по фото, деталям фигуры, походки, прочее
    - поиск музыки по фрагментам
    - "антиплагиат"
- обнаружение новых трендов и тем для обсуждения в социальных медиа и в интернете

Обычно эта задача решается при помощи алгоритма kNN - поиска $k$ ближайших соседей. Некоторые относят данный алгоритм в раздел "Обучение с учителем", но на самом деле, при каждом прогнозе данный алгоритм использует всю обучающую выборку. И поэтому вообще сомнительно утверждать то, что это имеет отношение к машинному обучению. Скорее всего, это дальнейшее развитие темы кластеризации данных.

Для алгоритма k-ближайших соседей (kNN) важен следующий препроцессинг данных:

1. **Обработка пропущенных значений**: Заполните или удалите пропущенные значения, чтобы алгоритм мог корректно вычислять расстояния.
2. **Преобразование категориальных данных**: Конвертируйте категориальные признаки в числовые для возможности вычисления расстояний.
3. **Удаление выбросов**: Избавьтесь от аномальных значений, которые могут исказить результаты.
3. **Масштабирование признаков**: Унифицируйте масштаб всех признаков, чтобы один не доминировал над другими. Можно использовать ```MinMaxScaler()```, ```StandardScaler()``` или ```normalization()```.
5. **Уменьшение размерности**: Сократите количество признаков, чтобы избежать проблемы "проклятия размерности".

Также на качество работы kNN влияет __выбор метрики расстояния__ (Евклидово, Косинусное, Хэмминга, пр.). Вот наиболее часто используемые метрики:

1. **Евклидово расстояние**: Идеально подходит для непрерывных числовых данных с одинаковым масштабом. 

2. **Расстояние Хэмминга**: Расстояние Хэмминга подсчитывает число позиций, в которых соответствующие символы двух строк различны. Хорошо работает с категориальными или бинарными данными. Пример: поиск изображений по хэш-значениям.

3. **Косинусное сходство**: Косинусное сходство измеряет угол между двумя векторами в многомерном пространстве, что позволяет судить о схожести по направлению, а не по магнитуде векторов. Пример: анализ текстов, где данные представлены в виде векторов слов или фраз.


Рассмотрим на примере датасета "Рукописные цифры".

In [ ]:
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler

digits = datasets.load_digits()
X, y = digits.data, digits.target

X = MinMaxScaler().fit_transform(X)

ix_random_image = 15

plt.figure(figsize=(4,2))
plt.imshow(X[ix_random_image].reshape((8, 8)), cmap='gray_r')
plt.xticks([])
plt.yticks([])
plt.title('Source image')
plt.show()

In [ ]:
from sklearn.neighbors import NearestNeighbors

neighbors = NearestNeighbors(n_neighbors=10,
                             algorithm='brute',
                             metric='euclidean')

neighbors.fit(X)

distances, indices = neighbors.kneighbors(X[ [ix_random_image] ])
print(distances)
print(indices)

In [ ]:
fig = plt.figure(figsize=(20, 2))
for i, (idx, digit) in enumerate(zip(indices[0], X[ indices[0] ])):
    ax = fig.add_subplot(1, 10, i+1, xticks=[], yticks=[])
    ax.imshow(digit.reshape((8, 8)), cmap='gray_r')
    ax.set_title(f"idx={idx}")
    
plt.show()

Классификатор ```KNeighborsClassifier``` может предсказывать и класс, и вероятность отнесения к классу.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                   test_size=0.2, 
                                                   random_state=20231110,
                                                   stratify = y)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score


model = KNeighborsClassifier(n_neighbors=10,
                             algorithm='brute',
                             metric='euclidean')

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)

print(f"Accuracy score on test set: {accuracy_score(y_test, y_pred)}")
print(f"ROC-AUC score on test set: {roc_auc_score(y_test, y_pred_proba, multi_class='ovr')}")

#### ⁉️ Задание

Преобразуйте данные в 2D методами главных компонент и t-SNE. Выполните кросс-валидацию классификатора ```KNeighborsClassifier``` на этих множествах для метрик ```accuracy_score``` и ```roc_auc```. Сделайте выводы.

In [ ]:
# ваш код здесь

